# Heart Failure Prediction

In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing Libraries** <a id="1"></a>

In [53]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.gridspec as grid_spec
import seaborn as sns

import scikitplot as skplt
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

# **Importing Dataset** <a id="2"></a>

In [54]:
df = pd.read_csv('../input/heart-failure-prediction/heart.csv')

In [55]:
df.head()

In [56]:
df.info()

In [57]:
df.isnull().sum()

There are no null values.

In [58]:
df.describe()

In [59]:
df.corr()

# EDA

## **Heat Map Correlation** <a id="3.1"></a>

In [60]:
# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

## **Count Plot** <a id="3.2"></a>

### **Sex** <a id="3.2.1"></a>

In [61]:
print(df.Sex.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="Sex")
plt.show()

You can see there are more Males present in our dataset than females.

## ChestPainType （狭心症）

In [62]:
print(df.ChestPainType.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="ChestPainType")
plt.show()

ASY >>> NAP > ATA >> TA.　(TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic)

## FastingBS （空腹時血糖値）

In [63]:
print(df.FastingBS.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="FastingBS")
plt.show()

Most patients have low fasting blood sugar (1: if FastingBS > 120 mg/dl, 0: otherwise)

## RestingECG （心電図検査）

In [64]:
print(df.RestingECG.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="RestingECG")
plt.show()

RestingECG : Resting electrocardiogram results （Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria）

## ExerciseAngina （労作性狭心）

In [65]:
print(df.ExerciseAngina.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="ExerciseAngina")
plt.show()

Most patients don't have ExerciseAngina(exercise-induced angina)
(Y: Yes, N: No)

## ST_Slope （心電図のSTセグメントのスロープ形状）

In [66]:
print(df.ST_Slope.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="ST_Slope")
plt.show()

ST_Slope: the slope of the peak exercise ST segment (Up: upsloping, Flat: flat, Down: downsloping)

An Up ST segment is one sign of acute myocardial infarction, while a Down ST segment is one sign of ischemia.

STセグメントの上昇は急性心筋梗塞、低下は虚血の1つの徴候である。

## HeartDisease

In [67]:
print(df.HeartDisease.value_counts())
sns.set_theme(style="darkgrid")
ax = sns.countplot(data=df, x="HeartDisease")
plt.show()

This will be the target variable. Basically balanced.

## **Distribution Plot** <a id="3.3"></a>

### RestingBP　（安静時血圧）

In [68]:
fig = plt.figure(figsize=(7,7))
sns.distplot(df.RestingBP, color="red", label="RestingBP", kde= True)
plt.legend()

resting blood pressure (mm Hg)　

In healthy individuals, resting blood pressure fluctuations are within a nearly constant range, with a maximum blood pressure of 100-140 and a minimum blood pressure of 60-90 mmHg at rest.

### Cholesterol （血清コレステロール）

In [69]:
fig = plt.figure(figsize=(7,7))
sns.distplot(df.Cholesterol, color="green", label="Cholesterol", kde= True)
plt.legend()

serum cholesterol (mm/dl)

Serum cholesterol refers to cholesterol dissolved in serum as lipoproteins, and the average serum cholesterol level is around 200 mg/dl in adults.

### MaxHR（最大心拍数）

In [70]:
fig = plt.figure(figsize=(7,7))
sns.distplot(df.MaxHR, color="green", label="MaxHR", kde= True)
plt.legend()

MaxHR: maximum heart rate achieved (Numeric value between 60 and 202)

The maximum heart rate can generally be obtained by "maximum heart rate = 220 - age". 

### **No HeartDisease vs HeartDisease by RestingBP** <a id="3.3.3"></a>

In [71]:
plt.figure(figsize=(10,8))

sns.distplot(df[df['HeartDisease'] == 0]["RestingBP"], color='green') # No HeartDisease - green
sns.distplot(df[df['HeartDisease'] == 1]["RestingBP"], color='red') # HeartDisease - Red

plt.title('No HeartDisease vs HeartDisease by RestingBP', fontsize=15)
plt.xlim([10,300])
plt.show()

### **No HeartDisease vs HeartDisease by Cholesterol** <a id="3.3.4"></a>

In [72]:
plt.figure(figsize=(10,8))

sns.distplot(df[df['HeartDisease'] == 0]["Cholesterol"], color='green') # No HeartDisease - green
sns.distplot(df[df['HeartDisease'] == 1]["Cholesterol"], color='red') # HeartDisease - Red

plt.title('No HeartDisease vs HeartDisease by Cholesterol', fontsize=15)
plt.xlim([0,700])
plt.show()

### **No HeartDisease vs HeartDisease by MaxHR** <a id="3.3.5"></a>

In [73]:
plt.figure(figsize=(10,8))

sns.distplot(df[df['HeartDisease'] == 0]["MaxHR"], color='green') # No HeartDisease - green
sns.distplot(df[df['HeartDisease'] == 1]["MaxHR"], color='red') # HeartDisease - Red

plt.title('No HeartDisease vs HeartDisease by MaxHR', fontsize=15)
plt.xlim([0,300])
plt.show()

### **No HeartDisease vs HeartDisease by Age** <a id="3.3.5"></a>

In [74]:
plt.figure(figsize=(10,8))

sns.distplot(df[df['HeartDisease'] == 0]["Age"], color='green') # No HeartDisease - green
sns.distplot(df[df['HeartDisease'] == 1]["Age"], color='red') # HeartDisease - Red

plt.title('No HeartDisease vs HeartDisease by Age', fontsize=15)
plt.xlim([15,90])
plt.show()

## **Scatter Plot** <a id="3.4"></a>

### **Age vs RestingBP** <a id="3.4.1"></a>

In [75]:
fig = plt.figure(figsize=(7,7))
graph = sns.scatterplot(data=df, x="Age", y="RestingBP", hue='Sex')
graph.axhline(y= 120, linewidth=4, color='r', linestyle= '--')
plt.show()

### **Age vs Cholesterol** <a id="3.4.2"></a>

In [76]:
fig = plt.figure(figsize=(7,7))
graph = sns.scatterplot(data=df, x="Age", y="Cholesterol", hue='Sex')
graph.axhline(y= 200, linewidth=4, color='r', linestyle= '--')
plt.show()

the average serum cholesterol level is around 200 mg/dl in adults.

### **Age vs MaxHR** <a id="3.4.2"></a>

In [77]:
fig = plt.figure(figsize=(7,7))
graph = sns.scatterplot(data=df, x="Age", y="MaxHR", hue='Sex')
plt.show()

MaxHR: maximum heart rate achieved (Numeric value between 60 and 202)

The maximum heart rate can generally be obtained by "maximum heart rate = 220 - age". So we can notice from the figure that the maximum heart rate becomes lower as people get older.

## **Violin Plot** <a id="3.5"></a>

In [78]:
plt.figure(figsize=(13,13))
sns.set_theme(style="darkgrid")
plt.subplot(2,3,1)
sns.violinplot(x = 'Sex', y = 'HeartDisease', data = df)
plt.subplot(2,3,2)
sns.violinplot(x = 'ChestPainType', y = 'HeartDisease', data = df)
plt.subplot(2,3,3)
sns.violinplot(x = 'FastingBS', y = 'HeartDisease', data = df)
plt.subplot(2,3,4)
sns.violinplot(x = 'RestingECG', y = 'HeartDisease', data = df)
plt.subplot(2,3,5)
sns.violinplot(x = 'ExerciseAngina', y = 'HeartDisease', data = df)
plt.xticks(fontsize=9, rotation=45)
plt.subplot(2,3,6)
sns.violinplot(x = 'ST_Slope', y = 'HeartDisease', data = df)
plt.show()

# **Data Preprocessing** <a id="4"></a>

In [79]:
df.head()

In [80]:
# define our features 
X = df.drop(["HeartDisease"], axis=1)
# define our target
y = df[["HeartDisease"]]

In [81]:
X.head()

In [82]:
y.head()

# Encoding <a id="5"></a>

## **Categorical Encoding** <a id="5.1"></a>

We are using **OneHotEncoder()** to encode the categorical columns: 'ChestPainType', 'RestingECG', 'ST_Slope'.

In [83]:
X_encoded = pd.get_dummies(X, columns=['ChestPainType', 'RestingECG', 'ST_Slope'])
X_encoded.head()

## Label Encoding <a id="5.2"></a>

We are using **LabelEncoder()** to encode binary columns: 'Sex', 'ExerciseAngina'.

In [84]:
from sklearn.preprocessing import LabelEncoder

columns = ['Sex', 'ExerciseAngina']

for column in columns:
    le = LabelEncoder()
    le.fit(X_encoded[column])
    X_encoded[column] = le.transform(X_encoded[column])

In [85]:
X_encoded.head()

Look, 'Male' in 'Sex' column were converted to 1 and 'Female' to 0. 

Likewise, 'N' in 'ExerciseAngina' column were converted to 0 and 'Y' to 1.

In [86]:
print('Shape of X_encoded: ', X_encoded.shape)
print('Shape of y: ', y.shape)

# Splitting the dataset into the Training set and Test set <a id="6"></a>

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size= 0.2, random_state= 0)

In [88]:
print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

# Model Selection <a id="9"></a>

In [89]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score

### Random Forest Classifier

In [90]:
rfc = RandomForestClassifier(n_estimators=50, max_depth=5, max_features='sqrt', random_state=0) 
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
roc_auc_score(y_test, y_pred)

In [91]:
print('confusion matrix = \n', confusion_matrix(y_test, y_pred, labels=[1, 0]))
print('accuracy = ', accuracy_score(y_test, y_pred))
print('precision = ', precision_score(y_test, y_pred))
print('recall = ', recall_score(y_test, y_pred))
print('f1 score = ', f1_score(y_test, y_pred))

# Model Interpretation

I'll use some valuable tools which help to uncover the supposed "Black Box" of machine learning algorithms.

Sometimes the models we create need to be explainable to stakeholders. 

In [92]:
# great resource: https://www.kaggle.com/dansbecker/advanced-uses-of-shap-values
import shap  

explainer = shap.TreeExplainer(rfc)

# calculate shap values. This is what we will plot.
shap_values = explainer.shap_values(X_test)

In [93]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

First, we plotted the contribution of each feature variable to the target variable.
We see that the contribution of ST_Slope_Up and ChestPainType_ASY to heart failure is high.

In [94]:
shap.summary_plot(shap_values[1], X_test)

## SHAP explained 
The plot above shows the effect of each feature on our predictions. The horizontal axis shows whether each feature value contributes to a higher or lower prediction value (Red indicates positive feature values and blue indicates negative feature values.) The vertical axis, on the other hand, shows how high the contribution of each feature variable is. For example, it is noticeable from the plot that the larger (right side) the target variable is, the bluer the MaxHR (Maximum heart rate) distribution becomes, which means the target variable and the MaxHR are negatively correlated. Likewise, we can also notice that  the target variable and the Oldpeak are positively correlated.

## There's more? SHAP dependence plots 
It is also interesting to see how the impact of each variable changes as the variable itself changes. The figure below shows how the impact of the Age variable changes as the Age variable itself changes. We can notice that when the Age variable increases, the SHAP value also increases - pushing the patient closer to our 1 condition (heart disease). This is also shown with color - red color representing those who suffered a stroke. 

In [95]:
shap.dependence_plot('Age', shap_values[1], X_test, interaction_index="Age",cmap=cmap,alpha=0.8,show=False)
plt.title("Age dependence plot",loc='left',fontfamily='serif',fontsize=15)
plt.ylabel("SHAP value for the 'Age' feature")
plt.show()

# LIME
When it comes to model interpretation, sometimes it is useful to unpack and focus on one example at a time.

The LIME package enables just that.

Lime stands for Local Interpretable Model-agnostic Explanations - here's an example:

In [96]:
X_test.head()

In [97]:
y_pred[:10]

In [98]:
# import lime
import lime.lime_tabular

# LIME has one explainer for all the models
explainer = lime.lime_tabular.LimeTabularExplainer(training_data=X_train.values, feature_names=X_train.columns.values.tolist(),
                                                  class_names=['NonHeartDisease', 'HeartDisease'], verbose=True, mode='classification')

In [99]:
# Choose the jth instance and use it to predict the results for that selection
j = 6
exp = explainer.explain_instance(X_test.values[j], rfc.predict_proba, num_features=10)

# Show the predictions
exp.show_in_notebook(show_table=True)